In [17]:
import pandas as pd
java = pd.read_csv('./java/java_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
java_user_count = java.groupby('user_id').count().reset_index()[['user_id','item_id']]
java_inter = java[java['user_id'].isin(java_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()
java_inter['time_avg'] = java.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
java_inter['time_var'] = java.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
java_inter['time_max'] = java.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
java_inter['time_min'] = java.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = java_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['scene'] = predict_y

beh_count = java.groupby(['user_id','behavior']).count().reset_index()
java_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
java_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
java_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
java_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
java_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
java_inter = java_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = java_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['cat'] = predict_y

alldata = []
for user in java_inter['user_id'].tolist():
    # print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==user]['item_id'].item().tolist()
    scene = java_inter[java_inter['user_id']==user]['scene'].item()
    cat = java_inter[java_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
java_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[305],2,3,172,[172]
1,1,[305],2,3,54,"[172, 54]"
2,1,[305],2,3,314,"[172, 54, 314]"
3,1,[305],2,3,1,"[172, 54, 314, 1]"
4,1,[305],2,3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...,...
10491,183,[300],2,1,1,"[363, 1]"
10492,183,[300],2,1,294,"[363, 1, 294]"
10493,183,"[300, 363]",2,1,1,[1]
10494,183,"[300, 363]",2,1,294,"[1, 294]"


In [18]:
java_new[java_new['pred_path'].apply(lambda x: len(x))<=5]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[305],2,3,172,[172]
1,1,[305],2,3,54,"[172, 54]"
2,1,[305],2,3,314,"[172, 54, 314]"
3,1,[305],2,3,1,"[172, 54, 314, 1]"
4,1,[305],2,3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...,...
10491,183,[300],2,1,1,"[363, 1]"
10492,183,[300],2,1,294,"[363, 1, 294]"
10493,183,"[300, 363]",2,1,1,[1]
10494,183,"[300, 363]",2,1,294,"[1, 294]"


In [19]:
java_new[java_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[305],2,3,172,[172]
1,1,[305],2,3,54,"[172, 54]"
2,1,[305],2,3,314,"[172, 54, 314]"
3,1,[305],2,3,1,"[172, 54, 314, 1]"
4,1,[305],2,3,350,"[172, 54, 314, 1, 350]"
...,...,...,...,...,...,...
10491,183,[300],2,1,1,"[363, 1]"
10492,183,[300],2,1,294,"[363, 1, 294]"
10493,183,"[300, 363]",2,1,1,[1]
10494,183,"[300, 363]",2,1,294,"[1, 294]"


In [20]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.16)
java_train = java_new[~java_new.index.isin(java_test)]
java_test

,user_id,inter_seq,scene,cat,target,pred_path
5201,88,[294],3,0,302,"[295, 35, 297, 298, 54, 300, 301, 302]"
4643,72,"[295, 350, 35, 297, 308, 119, 357, 361, 363, 2...",1,0,351,"[173, 1, 336, 351]"
10001,158,"[294, 295, 35]",1,1,119,"[297, 300, 301, 302, 303, 305, 306, 307, 363, ..."
4049,68,"[294, 295, 35, 297, 300, 301, 305, 310, 311, 3...",3,0,334,"[203, 334]"
4081,68,"[294, 295, 35, 297, 300, 301, 305, 310, 311, 3...",3,0,351,"[338, 210, 341, 1, 336, 350, 351]"
...,...,...,...,...,...,...
1413,25,"[332, 294]",0,1,310,"[298, 183, 331, 341, 361, 336, 310]"
1988,29,"[1, 363, 294, 295, 311, 35, 298, 297, 301, 302...",1,0,306,[306]
8208,95,"[1, 341, 294, 295, 35, 297, 298]",3,2,301,"[300, 301]"
1304,21,[321],0,0,172,"[322, 323, 324, 172]"


In [21]:
java_test.to_csv('java/java_test_10.csv',header=True,sep='\t',index=False)
java_train.to_csv('java/java_train_10.csv',header=True,sep='\t',index=False)

In [22]:
import pandas as pd

net = pd.read_csv('./net/net_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
net_user_count = net.groupby('user_id').count().reset_index()[['user_id','item_id']]
net_inter = net[net['user_id'].isin(net_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

net_inter['time_avg'] = net.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
net_inter['time_var'] = net.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
net_inter['time_max'] = net.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
net_inter['time_min'] = net.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
net_inter['inter_len'] = net_inter['item_id'].apply(lambda x : len(x))
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = net_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['scene'] = predict_y

beh_count = net.groupby(['user_id','behavior']).count().reset_index()
net_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
net_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
net_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
net_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
net_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = net_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['cat'] = predict_y

alldata = []
for user in net_inter['user_id'].tolist():
    # print(user)
    seq_len = len(net_inter[net_inter['user_id']==user]['item_id'].item())
    total_seq = net_inter[net_inter['user_id']==user]['item_id'].item().tolist()
    scene = net_inter[net_inter['user_id']==user]['scene'].item()
    cat = net_inter[net_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
net_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
net_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,2,[1],1,1,297,[297]
1,2,[1],1,1,291,"[297, 291]"
2,2,[1],1,1,286,"[297, 291, 286]"
3,2,[1],1,1,285,"[297, 291, 286, 285]"
4,2,"[1, 297]",1,1,291,[291]
...,...,...,...,...,...,...
12067,245,[285],2,0,291,"[286, 291]"
12068,245,"[285, 286]",2,0,291,[291]
12069,246,[285],2,1,286,[286]
12070,246,[285],2,1,287,"[286, 287]"


In [23]:
net_new[net_new['pred_path'].apply(lambda x: len(x))<=5]

,user_id,inter_seq,scene,cat,target,pred_path
0,2,[1],1,1,297,[297]
1,2,[1],1,1,291,"[297, 291]"
2,2,[1],1,1,286,"[297, 291, 286]"
3,2,[1],1,1,285,"[297, 291, 286, 285]"
4,2,"[1, 297]",1,1,291,[291]
...,...,...,...,...,...,...
12067,245,[285],2,0,291,"[286, 291]"
12068,245,"[285, 286]",2,0,291,[291]
12069,246,[285],2,1,286,[286]
12070,246,[285],2,1,287,"[286, 287]"


In [24]:
net_new[net_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,2,[1],1,1,297,[297]
1,2,[1],1,1,291,"[297, 291]"
2,2,[1],1,1,286,"[297, 291, 286]"
3,2,[1],1,1,285,"[297, 291, 286, 285]"
4,2,"[1, 297]",1,1,291,[291]
...,...,...,...,...,...,...
12067,245,[285],2,0,291,"[286, 291]"
12068,245,"[285, 286]",2,0,291,[291]
12069,246,[285],2,1,286,[286]
12070,246,[285],2,1,287,"[286, 287]"


In [26]:
net_test = net_new[net_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.15)
net_train = net_new[~net_new.index.isin(net_test)]
net_test

,user_id,inter_seq,scene,cat,target,pred_path
2211,72,"[1, 285, 295, 296, 297, 330, 299, 300, 301]",3,0,306,"[302, 303, 304, 305, 306]"
7148,140,"[293, 294, 295, 299, 1, 301]",3,0,305,"[290, 308, 297, 305]"
1178,36,[1],1,0,291,"[285, 291]"
1083,30,"[286, 289, 287, 298]",2,0,288,"[1, 303, 288]"
8775,180,"[1, 286, 287, 288, 293, 294, 303]",3,2,307,"[301, 305, 306, 307]"
...,...,...,...,...,...,...
6777,129,"[320, 331, 315, 328, 325, 326, 317, 318, 336, ...",0,0,310,"[309, 310]"
5369,120,"[1, 299, 297, 298, 294, 295, 301, 303, 325, 32...",1,4,308,[308]
11321,216,"[285, 286, 288, 289, 290, 291, 293, 292, 296, ...",3,0,294,"[325, 322, 323, 324, 287, 314, 316, 317, 319, ..."
378,5,"[1, 298, 290]",1,0,286,[286]


In [27]:
net_test.to_csv('net/net_test_10.csv',header=True,sep='\t',index=False)
net_train.to_csv('net/net_train_10.csv',header=True,sep='\t',index=False)

In [28]:
import pandas as pd

os = pd.read_csv('./os/os_mb_KG.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
os_user_count = os.groupby('user_id').count().reset_index()[['user_id','item_id']]
os_inter = os[os['user_id'].isin(os_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

os_inter['time_avg'] = os.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
os_inter['time_var'] = os.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
os_inter['time_max'] = os.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
os_inter['time_min'] = os.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
os_inter['inter_len'] = os_inter['item_id'].apply(lambda x : len(x))
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = os_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['scene'] = predict_y

beh_count = os.groupby(['user_id','behavior']).count().reset_index()
os_inter['done_count'] = beh_count[beh_count['behavior']=='done']['item_id']
os_inter['watch_count'] = beh_count[beh_count['behavior']=='watch']['item_id']
os_inter['enter_count'] = beh_count[beh_count['behavior']=='enter']['item_id']
os_inter['tag_count'] = beh_count[beh_count['behavior']=='tag']['item_id']
os_inter['dur_count'] = beh_count[beh_count['behavior']=='dur']['item_id']
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = os_inter[['done_count','watch_count','enter_count','tag_count','dur_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['cat'] = predict_y

alldata = []
for user in os_inter['user_id'].tolist():
    # print(user)
    seq_len = len(os_inter[os_inter['user_id']==user]['item_id'].item())
    total_seq = os_inter[os_inter['user_id']==user]['item_id'].item().tolist()
    scene = os_inter[os_inter['user_id']==user]['scene'].item()
    cat = os_inter[os_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
os_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
os_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[397],1,0,428,[428]
1,1,[397],1,0,429,"[428, 429]"
2,1,"[397, 428]",1,0,429,[429]
3,3,[397],2,0,400,[400]
4,3,[397],2,0,30,"[400, 30]"
...,...,...,...,...,...,...
23164,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,405,[405]
23165,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,495,"[405, 495]"
23166,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1,0,495,[495]
23167,302,[397],1,1,407,[407]


In [29]:
os_new[os_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[397],1,0,428,[428]
1,1,[397],1,0,429,"[428, 429]"
2,1,"[397, 428]",1,0,429,[429]
3,3,[397],2,0,400,[400]
4,3,[397],2,0,30,"[400, 30]"
...,...,...,...,...,...,...
23164,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,405,[405]
23165,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,495,"[405, 495]"
23166,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1,0,495,[495]
23167,302,[397],1,1,407,[407]


In [30]:
os_new[os_new['pred_path'].apply(lambda x: len(x))<=5]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[397],1,0,428,[428]
1,1,[397],1,0,429,"[428, 429]"
2,1,"[397, 428]",1,0,429,[429]
3,3,[397],2,0,400,[400]
4,3,[397],2,0,30,"[400, 30]"
...,...,...,...,...,...,...
23164,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,405,[405]
23165,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409]",1,0,495,"[405, 495]"
23166,301,"[401, 398, 476, 412, 485, 486, 492, 436, 409, ...",1,0,495,[495]
23167,302,[397],1,1,407,[407]


In [31]:
os_test = os_new[os_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.18)
os_train = os_new[~os_new.index.isin(os_test)]
os_test

,user_id,inter_seq,scene,cat,target,pred_path
14053,144,"[397, 6, 16, 38, 45, 50, 400, 29, 47, 174, 59]",0,2,82,[82]
7188,41,"[397, 42, 420, 90, 74, 49, 30, 400, 113, 111, ...",0,0,32,"[6, 116, 118, 38, 162, 163, 32]"
23029,274,"[397, 37, 40, 30, 16, 406, 412, 415]",0,0,413,"[410, 413]"
9033,83,"[397, 38, 40, 90, 45, 48, 49, 47, 50, 60, 44, ...",0,0,406,"[431, 444, 455, 452, 476, 399, 406]"
184,4,"[397, 398, 399, 400, 401, 402, 407, 410, 411, ...",1,0,409,"[414, 416, 403, 404, 405, 406, 408, 409]"
...,...,...,...,...,...,...
14002,144,"[397, 6, 16]",0,2,116,"[38, 45, 50, 400, 29, 47, 174, 59, 82, 116]"
23062,287,"[397, 398, 399]",2,2,405,"[400, 401, 402, 403, 404, 405]"
9052,83,"[397, 38, 40, 90, 45, 48, 49, 47, 50, 60, 44, ...",0,0,399,"[444, 455, 452, 476, 399]"
9220,83,"[397, 38, 40, 90, 45, 48, 49, 47, 50, 60, 44, ...",0,0,464,"[403, 404, 398, 428, 451, 439, 440, 464]"


In [32]:
os_test.to_csv('os/os_test_10.csv',header=True,sep='\t',index=False)
os_train.to_csv('os/os_train_10.csv',header=True,sep='\t',index=False)